In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# | hide
from onprem.core import *

# Few-Shot Classification



`FewShotClassifier` is a simple wrapper around the `SetFit` package and allows you to make text classification predictions on only a few labeled examples (e.g., 8 examples per class). It is useful when only a small amount of labeled examples are available for training a model. We will supply the `use_smaller=True` argument to use the smaller version of the default model.

In [ ]:
# | notest
from onprem.pipelines import FewShotClassifier
clf = FewShotClassifier(use_smaller=True)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


## STEP 1: Construct a Tiny Dataset

In this example, we will classify a sample of the 20NewsGroup dataset.

In [ ]:
# | notest

categories = ['comp.graphics', 'sci.med', 'sci.space', 'soc.religion.christian']
from sklearn.datasets import fetch_20newsgroups
train_b = fetch_20newsgroups(subset='train',
   categories=categories, shuffle=True, random_state=42)
test_b = fetch_20newsgroups(subset='test',
   categories=categories, shuffle=True, random_state=42)

X_train = train_b.data
y_train = [train_b.target_names[y] for y in train_b.target]
X_test = test_b.data
y_test = [test_b.target_names[y] for y in test_b.target]

# sample a small number of examples from full training set
X_sample, y_sample = clf.sample_examples(X_train, y_train, num_samples=8)

There are only 32 training examples!

In [ ]:
# | notest

len(X_sample)

32

There are 1502 test examples.

In [ ]:
# | notest

len(X_test)

1577

## STEP 2: Train on the Tiny Dataset

Let's train:

In [ ]:
# | notest

clf.train(X_sample,  y_sample, num_epochs=1, batch_size=16, num_iterations=20)

Applying column mapping to the training dataset


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 1280
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 80


Step,Training Loss


## STEP 3: Evaluate

After training, `model.labels` stores the labels.

In [ ]:
# | notest

print(clf.evaluate(X_test, y_test, print_report=True))

                        precision    recall  f1-score   support

         comp.graphics       0.92      0.92      0.92       389
               sci.med       0.93      0.90      0.92       396
             sci.space       0.90      0.87      0.88       394
soc.religion.christian       0.91      0.97      0.94       398

              accuracy                           0.92      1577
             macro avg       0.92      0.92      0.92      1577
          weighted avg       0.92      0.92      0.92      1577



**A 92% accuracy using only 32 examples!**

## STEP 4: Make Predictions on New Data

Let's make some predictions on new data:

In [ ]:
# | notest

clf.predict(['My grapics card sucks for machine learning.'])

array(['comp.graphics'], dtype='<U22')

In [ ]:
# | notest


clf.predict(['My mom likes going to church.'])

array(['soc.religion.christian'], dtype='<U22')

In [ ]:
# | notest
clf.predict(['SpaceX launches lots of satellites.'])

array(['sci.space'], dtype='<U22')

Show prediction probabilities:

In [ ]:
# | notest

clf.predict_proba(['SpaceX launches lots of satellites.'])

tensor([[0.1913, 0.1299, 0.5579, 0.1210]], dtype=torch.float64)

## STEP 5: Inspect and Explain Predictions

Explain predictions:

In [ ]:
# | notest

clf.explain(['My graphics card sucks for machine learning.'])

If you click on `comp.graphics`, you can see that the word "*graphics*" has the largest impact on the prediction after "*card*" and then "*machine*".

## STEP 6: Save and/or Reload the Model
Save and reload the model:

In [ ]:
# | notest

clf.save('/tmp/my_fewshot_model')

In [ ]:
# | notest

clf = FewShotClassifier('/tmp/my_fewshot_model')

In [ ]:
# | notest

clf.predict(['Elon Musk likes launching satellites.'])

array(['sci.space'], dtype='<U22')